In [1]:
%matplotlib inline


Core Concepts
=============

In [2]:
import pprint

 ``gensim``의 Core Concepts는 다음과 같습니다:

1. `core_concepts_document`: 몇가지 문장들.
2. `core_concepts_corpus`: document들의 모음.
3. `core_concepts_vector`: document를 수학적으로 편리하게 대응시킨 document.
4. `core_concepts_model`:  vector들에 한 representiation에서 다른 것으로 변환시키는 알고리즘.

## Document


Gensim 에서 *document* 는 Gensim의 document는 Python3의 str 객체입니다. document는 140자 짜리 트윗이 될수도 책이 될수도 있습니다.


In [3]:
document = "Human machine interface for lab abc computer applications"


Corpus
------

Gensim의 Corpus는 document가 모인 객체입니다. Corpora(Corpus의 복수형)는 Gensim에서 두가지 역할을 수행합니다.:

1. `core_concepts_model`의 입력값.
   모델학습에서 모델들은 내부적인 파라미터들을 초기화 시키면서 일반적인 테마와 토픽들을 찾는데에 corpus를 사용합니다.
   
   Gensim은 *비지도학습*에 초점을 맞추고 있기 때문에 사람의 간섭(어노테이션,태깅)이 필요하지 않습니다.

2. Document들을 체계화.
    
   모델 학습이 끝나면, 토픽 모델은 새로운 document에서 새로운 토픽을 추출해 내는데에 사용될 수 있습니다.

다음은 corpus의 예시입니다.

한개의 문자으로 된 stirng으로 구성된 9개의 ducument를 포함하고 있습니다.

In [4]:
text_corpus = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

중요한 사실:<br>
위의 예시들은 corpus 전체를 메모리상에 적재합니다.<br>
연습을 하면서, corpus 들이 너무 커져서 메모리에 올리는게 불가능 해질 수 있습니다.<br>
Gensim은 지능적으로 그런 corpus들을 *스트리밍* 해서 한번에 한 document를 다룹니다.<br>
<br>
여기선 특정한 목적으로 간단한 corpus들로 예시를 들었지만, 다른 예시에서는 셰익스피어의
작품일 수도 있고 위키피디아의 작성글이 될 수도 어떤사람의 관심사에 따른 트윗일 수도 있습니다<br>
<br>
corpus를 모았다면 우리가 감내해야 하는 일반적인 전처리들이 있습니다.<br>
여기선 간단히 불용어 처리를 하고, corpus에서 단 한번만 나온 단어를 제거해주겠습니다.<br>
이런 일련의 과정에서 우리는 데이터를 토큰화 합니다.<br>
<br>
토큰화는 document를 단어들로 쪼개어 놓습니다.

In [5]:
# 불용어 모으기
stoplist = set('for a of the and to in'.split(' '))

# 모든 document를 소문자로 바꾸고 공백으로 분리하며, 불용어를 걸러줍니다
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in text_corpus]

# 단어의 빈도를 계산합니다.
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# 빈도수가 1 이상인 토큰만 사용합니다.
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
pprint.pprint(processed_corpus)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


더 진행 하기전에 우리는 corpus에 있는 각 단어를 고유한 integer형 id로 변환하려고 합니다.<br>
이 동작은 `gensim.corpora.Dictionary` 클래스를 이용해서 할 수 있습니다.<br>
이 딕셔너리는 우리가 진행하면서 알아야할 모든 단어들을 정의해 놓습니다

In [6]:
from gensim import corpora

# 너무 느린거 아님......?
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [7]:
# 개선한 코드

from itertools import chain

word_unique = set(chain.from_iterable(processed_corpus))
new_dictionary = dict(zip(range(len(word_unique)),word_unique))
print(new_dictionary)

{0: 'minors', 1: 'interface', 2: 'survey', 3: 'response', 4: 'user', 5: 'eps', 6: 'graph', 7: 'computer', 8: 'system', 9: 'trees', 10: 'human', 11: 'time'}


corpus가 작기 때문에, 이 딕셔너리는 12개의 토큰 만 가지고 있지만, 더 긴 corpus들의 경우에는 백개, 천개가 넘는 토큰을 포함하는 것이 꽤 자주있는 일이다.


Vector
------

우리가 가진 corpus의 잠재된 구조를 추론하기위해 수학적으로 처리가 가능하게 document를 represent하는 방법이 필요하다.<br>
각 document를 represent하는 접근법 중에 하나가 바로 *features* 를 벡터화 하는 것이다.<br>
<br>
예를 들어, 한 feature는 질-답의 쌍으로 생각해볼수 있다.<br>
<br>
1. *splonge* 이라는 단어가 얼마나 자주 나왔나? : 0
2. document가 갖고 있는 단락의 갯수는 몇개 인가? : 2
3. document는 얼마나 다야한 폰트들을 사용하나? :5 <br>
<br>
<br>
이 질문들은 integer 형 id들에 대응 된다. 1번,2번,3번<br>
이 질문을 쌍으로 묶으면 (1,0),(2,2) (3,5) 처럼 나타낼 수 있다.<br>
이를 *dense vector* 라고 한다. 왜냐면 이 쌍들은 각 질문에 대한 explicit한 답을 가지고 있기 때문이다<br>
<br>
이 질문들에 대해 좀더 나아가면 아마도 implicit하게 놔둘수도 있을것이다.<br>
그리고 아마도 (0,2,5)처럼 document를 represent할 수 있을 것이다.<br>
<br>
이런 일련의 answer들이 우리의 document의 **vector** 이다 (이 경우에는 3차원 dense vector였다).<br>
<br>
좀더 실용적인 목적으로,  Gensim에서는 어떤 질문에 대해 32비트 float(또는 그것으로 변환될 수 있는 것)를 답으로 하는 것만 허용하고 있다.<br>
<br>
실제로, 벡터들은 주로 많은 zero 값으로 구성되어 있다.<br>
메모리를 알뜰하게 쓰기 위해 Gensim은 모든 0.0인 벡터 원소를 지운다.<br>
<br>
위의 예시는 실제론 ``(2, 2.0), (3, 5.0)``가 된다.<br>
<br>
이런 것들을 *sparse vector* 또는 *bag-of-words vector* 라고 부른다.<br>
<br>
이 희소행렬(sparse matrix)의 사라진 feature들은 명백히 zero가 된다.<br>
<br>
위에서 한 질문이 그대로 라는 가정하에 우리는 두가지 벡터를 서로 비교할 수 있다.<br>
예를 들어, ``(0.0, 2.0, 5.0)`` 와 ``(0.1, 1.9, 4.9)`` 가 주어졌다고 해보자<br>
vector들이 서로 매우 비슷하기 때문에 우리는 상응하는 두 벡터들이 서로 비슷하다고 생각할 수 있다.<br>
당연히도, 결과의 정확도는 얼마나 좋은 질문들을 처음에 골랐는지에 따라 달라진다.<br>

또다른 document를 vector로 만드는 접근법에는 *bag-of-words model*이 있다 <br>
<br>
bag-of-words 모델 하에서 각 document는 딕셔너리의 빈도수를 가지고 있는 vector로 대응된다.<br>
<br>
예를 들어 ``['coffee', 'milk', 'sugar', 'spoon']`` 이 단어로 있을때 <br>
``"coffee milk coffee"`` 라는 document는 ``[2, 1, 0, 0]`` 로 대응된다.<br>
벡터의 차원은 단어의 가짓수가 된다.<br>

In [8]:
pprint.pprint(dictionary.token2id)

{'computer': 0,
 'eps': 8,
 'graph': 10,
 'human': 1,
 'interface': 2,
 'minors': 11,
 'response': 3,
 'survey': 4,
 'system': 5,
 'time': 6,
 'trees': 9,
 'user': 7}


예를 들어, "Human computer interaction"이라는 구절을 벡터화 한다고 가정해보자. 우리는 희소한 형태로 단어의 빈도수를 매긴 딕셔너리의 doc2bow 메소드를 이용해서 bag-of-word를 생성 할 수 있다.

In [9]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 1), (1, 1)]


각 튜플의 첫번째 원소는 딕셔너리에서의 토큰의 ID, 두번째 요소는 토큰의 빈도수를 나타낸다.<br><br>
"interaction"이라는 단어는 먼저 갖고 있던 corpus에 한번도 등장하지 않았기 때문에 벡터화되지 않았다.<br><br>
이 벡터는 실제 document에서 등장한 단어들만 포함하고 있습니다.<br><br>
왜냐하면 document는 많은 단어중 일부의 단어만들 포함할 것이기 때문에 벡터화에 등장 하지 않은 단어들은 차원 감소를 통한 implicit한 zero 값으로 표현되기 때문입니다.<br>
<br>
우리는 원본 corpus 전체를 벡터의 list로 전환 할수 있습니다.

In [10]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


Model
-----

우리가 가진 corpus를 벡터화 했다면, 이제 model을 통해 corpus를 변환 시킬 수 있습니다.<br>
우리는 모델을 한 document representation을 다른 것으로 변환하는 작업을 지칭합니다.<br>

gensim에선 document들이 벡터들로 표현되어서 model이란게 두 벡터공간 사이에서 변환이라고 생각할 수 있습니다.<br>
model은 model을 학습하는 동안 이런 변환들의 세부사항에 대해서 학습을 합니다.<br>

가장 간단한 모델예시로는 tf-idf가 있습니다.

In [11]:
from gensim import models

# 모델학습
tfidf = models.TfidfModel(bow_corpus)

# system minors라는 단어를 학습
words = "system minors".lower().split()
print(tfidf[dictionary.doc2bow(words)])

[(5, 0.5898341626740045), (11, 0.8075244024440723)]


tfidf 모델은 튜플의 list를 반환합니다. 튜플은 (토큰의 ID, tf-idf 가중치)로 이루어져 있습니다.<br>
여기서 원본 corpus에선 4번 나온 system이 2번 나온 minors보다 tfidf 가중치가 낮게 나온것을 볼 수 있습니다.<br>
<br>
학습된 model은 저장했다가 나중에 다시 로드 할 수도 있고 새로운 document로 학습을 하거나 새로운 document를 변환 시킬수도 있습니다.<br>
<br>
`sphx_glr_auto_examples_core_run_topics_and_transformations.py`.을 보면 gensim에 제공하는 모델/변환 방식들이 있습니다.<br>
<br>
한번 model을 생성했다면, 모든 재미난 일들을 할 수 있습니다.<br>
예를들어 모든 corpus를 tfidf로 변환하고 인덱싱 했다면, 다음과 같은 유사도 쿼리를 할수 있습니다.<br>

In [12]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=12)

corpus에 있는 모든 document를 대상으로 유사도를 계산 할 수 있습니다

In [13]:
query_document = 'system engineering'.split()
query_bow = dictionary.doc2bow(query_document)
sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))

[(0, 0.0), (1, 0.32448703), (2, 0.41707572), (3, 0.7184812), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


3번 document가 약 72%의 유사도 점수를 갖습니다.
정렬 해보겠습니다.

In [14]:
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    print(document_number, score)

3 0.7184812
2 0.41707572
1 0.32448703
0 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0


Summary
-------

``gensim``의 Core Concepts는 다음과 같습니다:

1. `core_concepts_document`: 몇가지 문장들.
2. `core_concepts_corpus`: document들의 모음.
3. `core_concepts_vector`: document를 수학적으로 편리하게 대응시킨 document.
4. `core_concepts_model`:  vector들을 한 representation 에서 다른 것으로 변환시키는 알고리즘.
<br>
<br>

여기까지 콘셉트와 그들이 어떻게 작용하는지 보았습니다.<br>
첫째로, documents들의 corpus로 시작했고<br>
다음엔 이런 documents들을 벡터 공간으로 represent했고<br>
그후엔 원본 벡터를 변환시키는 tfidf 모델을 만들었습니다.<br>
<br>
최종적으로 모델을 이용해서 document와 corpus안의 모든 document 사이의 유사도를 구했습니다.<br>
<br>


What Next?
----------

`sphx_glr_auto_examples_core_run_corpora_and_vector_spaces.py`.에 대해서 공부 해보겠습니다.